In [2]:
import pandas as pd
import numpy as np
import json
# plotting
import matplotlib.pyplot as plt
#opening data
import os
import pathlib
import warnings

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [15]:
#Opening Dataset

path_players = os.path.join(str(pathlib.Path().resolve()),  'data', 'Wyscout', 'players.json')
path_event = os.path.join(str(pathlib.Path().resolve()),  'data', 'Wyscout', 'events_England.json')

with open(path_players) as f:
    players = json.load(f)
player_df = pd.DataFrame(players)

with open(path_event) as f:
    event = json.load(f)
train = pd.DataFrame(event)



In [16]:
#Prepare dataset

# -Filter events to only keep shots. 
# -We check for Son's ID in the player database
# -Look for the shots he made with right foot (id=402) and left foot (id=401)


#take shots only
shots = train.loc[train['subEventName'] == 'Shot']
#look for son's id
son_id = player_df.loc[player_df["shortName"] == "Son Heung-Min"]["wyId"].iloc[0]
#get son's shot
son_shots = shots.loc[shots["playerId"] == son_id]

#left leg shots
lefty_shots = son_shots.loc[son_shots.apply (lambda x:{'id':401} in x.tags, axis = 1)]
#right leg shots
righty_shots = son_shots.loc[son_shots.apply (lambda x:{'id':402} in x.tags, axis = 1)]

#create list with ones for left foot shots and -1 for right foot shots
l = [1] * len(lefty_shots)
l.extend([-1] * len(righty_shots))

In [17]:
from statsmodels.stats.descriptivestats import sign_test
test = sign_test(l, mu0 = 0)
pvalue = test[1]

if pvalue < 0.05:
    print("P-value amounts to", str(pvalue)[:5], "- We reject null hypothesis - Heung-Min Son is not ambidextrous")
else:
    print("P-value amounts to", str(pvalue)[:5], " - We do not reject null hypothesis - Heung-Min Son is ambidextrous")

P-value amounts to 0.142  - We do not reject null hypothesis - Heung-Min Son is ambidextrous
